In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from rouge import Rouge
from IPython.display import display

In [2]:
def setup_chrome_options():
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-notifications")
    options.add_argument("--blink-settings=imagesEnabled=false")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    options.page_load_strategy = 'eager'
    return options

def init_driver():
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=setup_chrome_options())
    driver.set_page_load_timeout(15)
    driver.set_script_timeout(5)
    return driver

def scrape_expert(driver, url):
    try:
        h1 = driver.find_element(By.TAG_NAME, 'h1').text
        content = ' '.join([x.text for x in driver.find_elements(By.CLASS_NAME, 'plain-text')])
        return {'url': url, 'text': f"{h1}\n{content}", 'success': True}
    except Exception:
        return {'url': url, 'error': "Expert scrape error", 'success': False}

def scrape_lenta(driver, url):
    try:
        h1 = driver.find_element(By.TAG_NAME, 'h1').text
        content = driver.find_element(By.CLASS_NAME, 'topic-body__content').text
        return {'url': url, 'text': f"{h1}\n{content}", 'success': True}
    except Exception:
        return {'url': url, 'error': "Lenta scrape error", 'success': False}

def scrape_ria(driver, url):
    try:
        h1 = driver.find_element(By.CLASS_NAME, 'article__title').text
        content = ' '.join([x.text for x in driver.find_elements(By.CLASS_NAME, 'article__text')])
        try:
            table = driver.find_element(By.CLASS_NAME, 'article__table').text
            return {'url': url, 'text': f"{h1}\n{content}\n{table}", 'success': True}
        except:
            return {'url': url, 'text': f"{h1}\n{content}", 'success': True}
    except Exception:
        return {'url': url, 'error': "RIA scrape error", 'success': False}

def scrape_vesti(driver, url):
    try:
        h1 = driver.find_element(By.TAG_NAME, 'h1').text
        content = driver.find_element(By.CLASS_NAME, 'js-mediator-article').text
        return {'url': url, 'text': f"{h1}\n{content}", 'success': True}
    except Exception:
        return {'url': url, 'error': "Vesti scrape error", 'success': False}

def scrape_generic(driver, url):
    try:
        h1 = driver.find_element(By.TAG_NAME, 'h1').text if driver.find_elements(By.TAG_NAME, 'h1') else ""
        body = driver.find_element(By.TAG_NAME, 'body').text
        return {'url': url, 'text': f"{h1}\n{body}", 'success': True}
    except Exception:
        return {'url': url, 'error': "Generic scrape error", 'success': False}

def process_url(url):
    driver = None
    try:
        driver = init_driver()
        driver.get(url)
        
        start_time = time.time()
        while time.time() - start_time < 2:
            try:
                if "expert.ru" in url:
                    return scrape_expert(driver, url)
                elif "lenta.ru" in url:
                    return scrape_lenta(driver, url)
                elif "ria.ru" in url:
                    return scrape_ria(driver, url)
                elif "vesti.ru" in url:
                    return scrape_vesti(driver, url)
                else:
                    return scrape_generic(driver, url)
            except Exception:
                time.sleep(0.5)
                continue
        
        return {'url': url, 'error': 'Timeout', 'success': False}
    except Exception as e:
        return {'url': url, 'error': str(e), 'success': False}
    finally:
        if driver:
            driver.quit()

def run_scraper(input_file, output_file, workers=4):
    df = pd.read_excel(input_file)
    urls = df.URL.tolist()
    original_columns = df.columns.tolist()
    
    results = [None] * len(urls)
    url_to_index = {url: idx for idx, url in enumerate(urls)}
    
    with ThreadPoolExecutor(max_workers=workers) as executor:
        futures = {executor.submit(process_url, url): url for url in urls}
        
        with tqdm(total=len(urls), desc="Parsing URLs") as pbar:
            for future in as_completed(futures):
                url = futures[future]
                idx = url_to_index[url]
                results[idx] = future.result()
                pbar.update(1)
    
    result_df = pd.DataFrame(results)
    
    for col in original_columns:
        if col != 'URL' and col in df.columns:
            result_df[col] = df[col].values
    
    result_df.to_csv(output_file, index=False)
    return result_df

# Запуск парсера
parsed_data = run_scraper(
    input_file='Test_check.xlsx',
    output_file='selenium_parsed_news.csv',
    workers=6
)

Parsing URLs: 100%|████████████████████████████████████████████████████████████████████| 50/50 [01:51<00:00,  2.23s/it]


In [6]:
def calculate_metrics(reference, extracted):
    """Расчет метрик качества между двумя текстами"""
    rouge = Rouge()
    reference = str(reference)
    extracted = str(extracted)
    
    # Базовые метрики
    ref_words = set(reference.lower().split())
    ext_words = set(extracted.lower().split())
    common_words = ref_words & ext_words
    
    recall = len(common_words) / len(ref_words) if ref_words else 0
    precision = len(common_words) / len(ext_words) if ext_words else 0
    
    # ROUGE метрики
    try:
        rouge_scores = rouge.get_scores(extracted, reference)[0]
        rouge_l = rouge_scores['rouge-l']['f']
        rouge_1 = rouge_scores['rouge-1']['f']
    except:
        rouge_l = rouge_1 = 0.0
    
    return {
        'recall': recall,
        'precision': precision,
        'rouge_l': rouge_l,
        'rouge_1': rouge_1
    }

def determine_importance(score):
    """Определение уровня важности на основе score"""
    if score < 0.6:
        return "Критическая"
    elif score < 0.8:
        return "Высокая"
    elif score < 0.9:
        return "Средняя"
    return "Низкая"

def evaluate_text_pair(reference, extracted):
    """Оценка пары текстов (эталонный и извлеченный)"""
    metrics = calculate_metrics(reference, extracted)
    
    # Рассчитываем общий score (среднее всех метрик)
    score = sum(metrics.values()) / len(metrics)
    
    return {
        'original_text': str(reference)[:200] + '...' if len(str(reference)) > 200 else str(reference),
        'score': round(score, 3),
        'importance': determine_importance(score)
    }

def analyze_dataframe(df, reference_col='text', extracted_col='lib_text'):
    """Анализ всего DataFrame с текстами"""
    results = []
    
    for _, row in df.iterrows():
        results.append(evaluate_text_pair(row[reference_col], row[extracted_col]))
    
    # Создаем DataFrame с результатами
    return pd.DataFrame(results)[['original_text', 'score', 'importance']]

# Загрузка данных
df = pd.read_csv('selenium_parsed_news.csv')
    
# Анализ и вывод результатов
result_df = analyze_dataframe(df)
    
print("Результаты оценки качества извлечения текста:")
display(result_df)
    
# Сохранение результатов
result_df.to_excel('text_quality_evaluation.xlsx', index=False)
print("\nРезультаты сохранены в text_quality_evaluation.xlsx")

Результаты оценки качества извлечения текста:


,original_text,score,importance
0,Выгодна ли России отмена санкций США\nНовая ко...,0.971,Низкая
1,"Ключевая ставка ЦБ РФ: для чего нужна, почему ...",0.958,Низкая
2,Денис Мантуров: «Государство в рамках нацпроек...,0.980,Низкая
3,"Самозапрет на кредиты: суть, как оформить, плю...",0.965,Низкая
4,Компании по всему миру врут об экологичности с...,0.973,Низкая
5,20 лет назад спецназ ликвидировал президента И...,0.897,Средняя
6,Инвесторов связывают блокчейном\nЦифровые фина...,0.956,Низкая
7,"Выплаты и льготы участникам СВО: какие есть, к...",0.951,Низкая
8,"Геннадий Красников: «У нас остались школы, дос...",0.974,Низкая
9,Политика конфиденциальности\nФедеральное госуд...,0.936,Низкая



Результаты сохранены в text_quality_evaluation.xlsx


In [12]:
a = pd.read_csv('selenium_parsed_news.csv')
n = 4
print(a.url[n])
print('--', len(a.text[n]), '--', a.text[n])
print('--', len(a.lib_text[n]), '--', a.lib_text[n])

https://lenta.ru/brief/2025/03/11/green/
-- 12990 -- Компании по всему миру врут об экологичности своих продуктов. Что такое гринвошинг и зачем бизнес обманывает людей?
1
Что такое гринвошинг?
Заходя в магазин, вы часто замечали зеленые упаковки товаров или надписи об их экологичности и полезности для здоровья. Впрочем, к приобретению подобных продуктов следует относиться с особой осторожностью. Возможно, вы столкнулись с гринвошингом — маркетинговой уловкой, убеждающей покупателя в том, что бренд или товар экологичен и полезен.
Сам термин родился благодаря слиянию двух английских слов: green («зеленый» в значении экологичный) и washing («отмывание» в значении отбеливание репутации).
Первым его использовал в 1986 году американский эколог Джей Вестервельд. Так он окрестил феномен, с которым познакомился в одном из отелей на Самоа в Океании. Постояльцев настоятельно просили лишний раз не пачкать постельное белье и беречь воду — якобы потому, что она была в дефиците, а стоки мгновенно заг